<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url
# assign the response to a object
import requests

# Perform an HTTP GET request to the Falcon 9 Launch Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful")
else:
    print(f"Request failed with status code {response.status_code}")

# Get the HTML content from the response
html_content = response.content

# Print the first 500 characters of the HTML content to verify
print(html_content[:500].decode('utf-8'))


Request was successful
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-appearance-disabled vector-feature-appearance-pinned-clie


Create a `BeautifulSoup` object from the HTML `response`


In [ ]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [ ]:
# Use soup.title attribute

In [6]:
import requests
from bs4 import BeautifulSoup

# Step 1: Perform an HTTP GET request to the Falcon 9 Launch Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful")
else:
    print(f"Request failed with status code {response.status_code}")

# Step 2: Get the HTML content from the response
html_content = response.content

# Step 3: Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Verify the BeautifulSoup object by printing its title
print(soup.title)


Request was successful
<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [7]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
import requests
from bs4 import BeautifulSoup

# Step 1: Perform an HTTP GET request to the Falcon 9 Launch Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful")
else:
    print(f"Request failed with status code {response.status_code}")

# Step 2: Get the HTML content from the response
html_content = response.content

# Step 3: Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Step 4: Use the find_all function to find all table elements
html_tables = soup.find_all('table')

# Verify the result by printing the number of tables found
print(f"Number of tables found: {len(html_tables)}")

# Optionally, print the first table to verify
print(html_tables[0].prettify()[:500])


Request was successful
Number of tables found: 15
<table class="wikitable plainrowheaders collapsible" id="2023ytd" style="width: 100%;">
 <tbody>
  <tr class="is-sticky">
   <th scope="col">
    Flight No.
   </th>
   <th scope="col">
    Date and
    <br/>
    time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     Version,
     <br/>
     booster
 


Starting from the third table is our target table contains the actual launch records.


In [8]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable" style="width:100%; font-size:1.00em; line-height:1.5em;">
<tbody><tr class="is-sticky">
<th scope="col" style="width: 10%;">Date and time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>booster</a><sup class="reference" id="cite_ref-booster_26-2"><a href="#cite_note-booster-26">[c]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_27-2"><a href="#cite_note-Dragon-27">[d]</a></sup>
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th></tr>
<tr>
<td rowspan="2">22 May 2024<br/>08:00
</td>
<td><a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">F9 B5</a> ♺<br/><a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1071" title="List of Falcon 9 first-stage boosters">B1071.16</a>
</td>
<td><a href="/wiki/Vandenber

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [11]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


In [12]:
import requests
from bs4 import BeautifulSoup

# Step 1: Perform an HTTP GET request to the Falcon 9 Launch Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful")
else:
    print(f"Request failed with status code {response.status_code}")

# Step 2: Get the HTML content from the response
html_content = response.content

# Step 3: Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Step 4: Use the find_all function to find all table elements
html_tables = soup.find_all('table')

# Assuming the first table is the one we are interested in
first_launch_table = html_tables[0]

# Provided function to extract column name from header
def extract_column_from_header(th):
    return th.get_text().strip()

# Step 5: Use find_all() function with 'th' element on first_launch_table
th_elements = first_launch_table.find_all('th')

# Step 6: Iterate through each th element, extract the column name, and append to column_names list
column_names = []
for th in th_elements:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

# Verify the column names
print("Extracted column names:", column_names)

Request was successful
Extracted column names: ['Flight No.', 'Date andtime (UTC)', 'Version,booster[c]', 'Launchsite', 'Payload[d]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '195', '196', 'FH 5', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', 'FH 6', '221', '222', '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '241', '242', 'FH 7', '243', '244', '245', '246', '247', '248', '249', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', 'FH 8', '263', '264', '265', '266', '267', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', 'FH 9', '285']


Check the extracted column names


In [13]:
print(column_names)

['Flight No.', 'Date andtime (UTC)', 'Version,booster[c]', 'Launchsite', 'Payload[d]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '195', '196', 'FH 5', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', 'FH 6', '221', '222', '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '241', '242', 'FH 7', '243', '244', '245', '246', '247', '248', '249', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', 'FH 8', '263', '264', '265', '266', '267', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', 'FH 9', '285']


## TASK 3: Create a data frame by parsing the launch HTML tables


In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Step 1: Perform an HTTP GET request to the Falcon 9 Launch Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful")
else:
    print(f"Request failed with status code {response.status_code}")

# Step 2: Get the HTML content from the response
html_content = response.content

# Step 3: Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Step 4: Use the find_all function to find all table elements
html_tables = soup.find_all('table', {'class': 'wikitable'})

# Assuming the first table is the one we are interested in
first_launch_table = html_tables[0]

# Provided function to extract column name from header
def extract_column_from_header(th):
    return th.get_text().strip()

# Step 5: Use find_all() function with 'th' element on first_launch_table
th_elements = first_launch_table.find_all('th')

# Step 6: Iterate through each th element, extract the column name, and append to column_names list
column_names = []
for th in th_elements:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

# Remove any duplicate column names
column_names = list(dict.fromkeys(column_names))

# Step 7: Extract rows from the table
rows = []
for tr in first_launch_table.find_all('tr')[1:]:  # Skip the header row
    cells = tr.find_all(['td', 'th'])
    row = [cell.get_text().strip() for cell in cells]
    # Ensure the row length matches the number of headers
    if len(row) == len(column_names):
        rows.append(row)

# Step 8: Create a DataFrame from the extracted data
df = pd.DataFrame(rows, columns=column_names)

# Display the first few rows of the DataFrame
print(df.head())

Request was successful
Empty DataFrame
Columns: [Flight No., Date andtime (UTC), Version,booster[c], Launchsite, Payload[d], Payload mass, Orbit, Customer, Launchoutcome, Boosterlanding, 195, 196, FH 5, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, FH 6, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, FH 7, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, FH 8, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, ...]
Index: []

[0 rows x 106 columns]


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Step 1: Perform an HTTP GET request to the Falcon 9 Launch Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful")
else:
    print(f"Request failed with status code {response.status_code}")

# Step 2: Get the HTML content from the response
html_content = response.content

# Step 3: Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Step 4: Use the find_all function to find all table elements
html_tables = soup.find_all('table', {'class': 'wikitable'})

# Assuming the first table is the one we are interested in
first_launch_table = html_tables[0]

# Provided function to extract column name from header
def extract_column_from_header(th):
    return th.get_text().strip()

# Step 5: Use find_all() function with 'th' element on first_launch_table
th_elements = first_launch_table.find_all('th')

# Step 6: Iterate through each th element, extract the column name, and append to column_names list
column_names = []
for th in th_elements:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

# Remove any duplicate column names
column_names = list(dict.fromkeys(column_names))

# Initialize an empty dictionary with keys from the extracted column names
data_dict = {col: [] for col in column_names}

# Step 7: Extract rows from the table
for tr in first_launch_table.find_all('tr')[1:]:  # Skip the header row
    cells = tr.find_all(['td', 'th'])
    row = [cell.get_text().strip() for cell in cells]
    # Ensure the row length matches the number of headers
    if len(row) == len(column_names):
        for i, col in enumerate(column_names):
            data_dict[col].append(row[i])
    else:
        # Handle the case where the number of columns in the row does not match the headers
        for i, col in enumerate(column_names):
            if i < len(row):
                data_dict[col].append(row[i])
            else:
                data_dict[col].append(None)

# Step 8: Create a DataFrame from the dictionary
df = pd.DataFrame(data_dict)

# Display the first few rows of the DataFrame
print(df.head())

Request was successful
                                          Flight No.  \
0                                                195   
1  Dedicated SmallSat Rideshare mission to Sun-sy...   
2                                                196   
3  Following the Russian invasion of Ukraine, One...   
4                                               FH 5   

         Date andtime (UTC)            Version,booster[c]    Launchsite  \
0   3 January 202314:56[21]          F9 B5 ♺ B1060.15[22]  CCSFS,SLC-40   
1                      None                          None          None   
2  10 January 202304:50[27]               F9 B5 ♺ B1076.2  CCSFS,SLC-40   
3                      None                          None          None   
4  15 January 202322:56[33]  Falcon Heavy B5 B1070 (core)    KSC,LC-39A   

                                          Payload[d]          Payload mass  \
0   Transporter-6: (115 payloads Smallsat Rideshare)            Unknown[e]   
1                                

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def date_time(cell):
    # Extract date and time from the cell
    return cell.get_text().strip().split(' ')

def booster_version(cell):
    # Extract booster version from the cell
    return cell.get_text().strip()

def get_mass(cell):
    # Extract mass from the cell
    return cell.get_text().strip()

def landing_status(cell):
    # Extract landing status from the cell
    return cell.get_text().strip()

# Step 1: Perform an HTTP GET request to the Falcon 9 Launch Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful")
else:
    print(f"Request failed with status code {response.status_code}")

# Step 2: Get the HTML content from the response
html_content = response.content

# Step 3: Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Initialize an empty dictionary with keys for each column
launch_dict = {
    'Flight No.': [],
    'Date': [],
    'Time': [],
    'Version Booster': [],
    'Launch Site': [],
    'Payload': [],
    'Payload mass': [],
    'Orbit': [],
    'Customer': [],
    'Launch outcome': [],
    'Booster landing': []
}

extracted_row = 0

# Extract each table 
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Get table row 
    for rows in table.find_all("tr"):
        # Check to see if first table heading is a number corresponding to launch number 
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        # Get table element 
        row = rows.find_all('td')
        # If it is a number, save cells in a dictionary 
        if flag:
            extracted_row += 1
            # Flight Number value
            launch_dict['Flight No.'].append(flight_number)
            
            # Date value
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            
            # Time value
            time = datatimelist[1] if len(datatimelist) > 1 else None
            launch_dict['Time'].append(time)
            
            # Booster version
            bv = booster_version(row[1])
            if not bv and row[1].a:
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            launch_site = row[2].a.string if row[2].a else None
            launch_dict['Launch Site'].append(launch_site)
            
            # Payload
            payload = row[3].a.string if row[3].a else None
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            orbit = row[5].a.string if row[5].a else None
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            customer = row[6].a.string if row[6].a else None
            launch_dict['Customer'].append(customer)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0] if row[7].strings else None
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

# Convert the dictionary to a DataFrame
df = pd.DataFrame(launch_dict)

# Display the first few rows of the DataFrame
print(df.head())

Request was successful
  Flight No. Date     Time       Version Booster Launch Site  \
0        195    3  January  F9 B5 ♺ B1060.15[22]       CCSFS   
1        196   10  January       F9 B5 ♺ B1076.2       CCSFS   
2        197   18  January       F9 B5 ♺ B1077.2       CCSFS   
3        198   19  January          F9 B5B1075.1        VSFB   
4        199   26  January        F9 B5 ♺B1067.9       CCSFS   

              Payload                Payload mass  Orbit Customer  \
0                None                  Unknown[e]    SSO     None   
1              OneWeb        6,000 kg (13,000 lb)  Polar   OneWeb   
2             USA-343         4,352 kg (9,595 lb)    MEO     USSF   
3  Starlink Group 2–4       15,000 kg (33,000 lb)    LEO   SpaceX   
4  Starlink Group 5-2  ~17,400 kg (38,400 lb)[46]    LEO   SpaceX   

  Launch outcome      Booster landing  
0      Success\n  Success(ground pad)  
1      Success\n  Success(ground pad)  
2      Success\n  Success(drone ship)  
3      Success\n 

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [24]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [25]:
# Export the DataFrame to a CSV file
df.to_csv('falcon9_launches.csv', index=False)
print("Data exported to falcon9_launches.csv")

Data exported to falcon9_launches.csv


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
